<a href="https://colab.research.google.com/github/honyango/Analog-World-Clock/blob/master/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Pipeline

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import os

# Mount Drive and load CSV (example for train)
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Alzheimers Detection Dataset/Alzheimers_Detection_dataset'
train_df = pd.read_csv(os.path.join(base_path, 'CSV_datafiles', '_train_classes.csv'))

class AlzheimerDataset(Dataset):
    def __init__(self, df, folder, transform=None):
        self.df = df
        self.folder = folder
        self.transform = transform
        self.base_path = base_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filename = self.df.iloc[idx]['filename']
        img_path = os.path.join(self.base_path, self.folder, filename)
        image = cv2.imread(img_path)

        if image is None:
            # Handle cases where image loading fails
            # For now, we'll raise an error, but in a real pipeline, you might want to skip or log.
            raise ValueError(f"Image not found or corrupted: {img_path}")

        # Ensure image is 3-channel. If grayscale (2D), convert to BGR first.
        if image.ndim == 2:  # Grayscale image
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        # Now 'image' is guaranteed to be 3-channel BGR, so convert to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = self.df.iloc[idx, 1:].values.argmax()  # Class index (for classification; adapt for mask)
        # Placeholder mask: Create with same dimensions as image
        mask = torch.zeros((image.shape[0], image.shape[1])) # Dynamically size mask to match image

        if self.transform:
            augmented = self.transform(image=image, mask=mask.numpy())
            image = augmented['image'] # This is now a torch.Tensor in CHW format
            mask = augmented['mask']   # This is now a torch.Tensor

        return image.float(), mask.long(), label # Directly return the tensors from augmented

# Transforms with albumentations (pip install albumentations)
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2()
])

train_dataset = AlzheimerDataset(train_df, 'train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model Instantiation

In [ ]:
import torch.nn as nn
import torch

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super(UNet, self).__init__()
        # Encoder: Conv blocks with maxpool
        self.enc1 = self.conv_block(n_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        # Placeholder for full UNet architecture. Let's complete a simple one for demonstration.
        self.enc2 = self.conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = self.conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = self.conv_block(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.enc5 = self.conv_block(512, 1024)

        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = self.conv_block(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = self.conv_block(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = self.conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = self.conv_block(128, 64)

        self.out = nn.Conv2d(64, n_classes, 1)

    def conv_block(self, in_c, out_c):
        return nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x) # 64
        p1 = self.pool1(e1)
        e2 = self.enc2(p1) # 128
        p2 = self.pool2(e2)
        e3 = self.enc3(p2) # 256
        p3 = self.pool3(e3)
        e4 = self.enc4(p3) # 512
        p4 = self.pool4(e4)
        e5 = self.enc5(p4) # 1024

        # Decoder
        d4 = self.up4(e5) # 512
        d4 = torch.cat((e4, d4), dim=1) # 1024
        d4 = self.dec4(d4) # 512

        d3 = self.up3(d4) # 256
        d3 = torch.cat((e3, d3), dim=1) # 512
        d3 = self.dec3(d3) # 256

        d2 = self.up2(d3) # 128
        d2 = torch.cat((e2, d2), dim=1) # 256
        d2 = self.dec2(d2) # 128

        d1 = self.up1(d2) # 64
        d1 = torch.cat((e1, d1), dim=1) # 128
        d1 = self.dec1(d1) # 64

        return torch.sigmoid(self.out(d1))

# Dynamically select device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = UNet().to(device) # Move model to selected device

Using device: cpu


Training Loop

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

for epoch in range(10):  # Example epochs
    model.train()
    for images, masks, _ in train_loader:
        # Move tensors to the selected device
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation similarly, compute metrics


    model.eval()

In [ ]:
from sklearn.metrics import jaccard_score  # Or custom Dice

model.eval()
with torch.no_grad():
    for images, masks, _ in test_loader:
        outputs = model(images.cuda())
        preds = (outputs > 0.5).float()
        # Compute Dice: 2 * (pred & gt).sum() / (pred.sum() + gt.sum())

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import os

# Mount Drive and load CSV (example for train)
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Alzheimers Detection Dataset/Alzheimers_Detection_dataset'
train_df = pd.read_csv(os.path.join(base_path, 'CSV_datafiles', '_train_classes.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'CSV_datafiles', '_test_classes.csv'))

class AlzheimerDataset(Dataset):
    def __init__(self, df, folder, transform=None):
        self.df = df
        self.folder = folder
        self.transform = transform
        self.base_path = base_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filename = self.df.iloc[idx]['filename']
        img_path = os.path.join(self.base_path, self.folder, filename)
        image = cv2.imread(img_path)

        if image is None:
            # Handle cases where image loading fails
            # For now, we'll raise an error, but in a real pipeline, you might want to skip or log.
            raise ValueError(f"Image not found or corrupted: {img_path}")

        # Ensure image is 3-channel. If grayscale (2D), convert to BGR first.
        if image.ndim == 2:  # Grayscale image
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        # Now 'image' is guaranteed to be 3-channel BGR, so convert to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = self.df.iloc[idx, 1:].values.argmax()  # Class index (for classification; adapt for mask)
        # Placeholder mask: Create with same dimensions as image
        mask = torch.zeros((image.shape[0], image.shape[1])) # Dynamically size mask to match image

        if self.transform:
            augmented = self.transform(image=image, mask=mask.numpy())
            image = augmented['image'] # This is now a torch.Tensor in CHW format
            mask = augmented['mask']   # This is now a torch.Tensor

        return image.float(), mask.long(), label # Directly return the tensors from augmented

# Transforms with albumentations (pip install albumentations)
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2()
])

train_dataset = AlzheimerDataset(train_df, 'train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataset = AlzheimerDataset(test_df, 'test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.metrics import jaccard_score  # Or custom Dice

model.eval()
with torch.no_grad():
    for images, masks, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = (outputs > 0.5).float()
        # Compute Dice: 2 * (pred & gt).sum() / (pred.sum() + gt.sum())

Model Evaluation

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Alzheimers Detection Dataset/Alzheimers_Detection_dataset'
test_df = pd.read_csv(base_path + '_test_classes.csv')
class_counts = test_df[['MD', 'MoD', 'ND', 'VMD']].sum()
print(class_counts)  # Outputs: MD 84, MoD 5, ND 319, VMD 230

# For metrics (post-training, with preds and gts as tensors)
from torchmetrics import Dice, JaccardIndex, Precision
dice = Dice().cuda()
iou = JaccardIndex(task='binary').cuda()
prec = Precision(task='binary').cuda()
# In eval loop: dice(preds, gts), etc.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Alzheimers Detection Dataset/Alzheimers_Detection_dataset_test_classes.csv'

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Alzheimers Detection Dataset/Alzheimers_Detection_dataset'

# --- Diagnosing the FileNotFoundError ---
# Let's list the contents of the directory to verify the file path and name.
# This line will help you see if '_test_classes.csv' or a similar file exists.
# import os
# print(os.listdir(base_path))

# Original line causing error:
# test_df = pd.read_csv(base_path + '_test_classes.csv')

# Assuming the file path is corrected, the rest of the code will follow.
# class_counts = test_df[['MD', 'MoD', 'ND', 'VMD']].sum()
# print(class_counts)  # Outputs: MD 84, MoD 5, ND 319, VMD 230

# For metrics (post-training, with preds and gts as tensors)

!pip install torchmetrics # Install the missing library

from torchmetrics.classification import Dice
from torchmetrics import JaccardIndex, Precision
dice = Dice().cuda()
iou = JaccardIndex(task='binary').cuda()
prec = Precision(task='binary').cuda()
# In eval loop: dice(preds, gts), etc.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 14.4 MB/s eta 0:00:00


ImportError: cannot import name 'Dice' from 'torchmetrics.classification' (/usr/local/lib/python3.12/dist-packages/torchmetrics/classification/__init__.py)